In [1]:
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate, ChatPromptTemplate
from langchain_openai import OpenAI, ChatOpenAI
import wikipedia
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain_community.embeddings.openai import OpenAIEmbeddings
from langchain.memory import ConversationBufferMemory

In [23]:
api_key = open('api_key.txt').read()
llm = OpenAI(api_key = api_key)

template = """"You are a helpful and compassionate chatbot explaining about mental health disorders to employees in the tech industry at potential risk.
They answered questions on a survey, and have been found to be at risk for {disorder}. 

Context: {disorder_context}

(if the context is irrelevant, ignore it and answer to the best of your abilities)

Information about the person: 
{survey}

Their Past Prompts: {past_qs}

Their Question: {question}

Answer: """

prompt = PromptTemplate.from_template(template)

In [ ]:
disorder_context = {}

for disorder in ['Anxiety Disorder', 'Mood Disorder']:
    text = ""
    pages = wikipedia.search(disorder)
    for p in pages:
        try: text += wikipedia.page(p).content + '\n\n'
        except: continue
        
    disorder_context[disorder] = text

In [ ]:
text_splitter = CharacterTextSplitter(chunk_size=350, 
                                      chunk_overlap=0, separator = '.')
chunked_texts = text_splitter.split_text(disorder_context['Anxiety Disorder'])

db = FAISS.from_texts(chunked_texts, OpenAIEmbeddings(api_key = api_key))
db.save_local('anxiety')

Created a chunk of size 489, which is longer than the specified 350
Created a chunk of size 446, which is longer than the specified 350
Created a chunk of size 368, which is longer than the specified 350
Created a chunk of size 417, which is longer than the specified 350
Created a chunk of size 392, which is longer than the specified 350
Created a chunk of size 1457, which is longer than the specified 350
Created a chunk of size 398, which is longer than the specified 350
Created a chunk of size 689, which is longer than the specified 350
Created a chunk of size 479, which is longer than the specified 350
Created a chunk of size 360, which is longer than the specified 350
Created a chunk of size 451, which is longer than the specified 350
Created a chunk of size 446, which is longer than the specified 350
Created a chunk of size 419, which is longer than the specified 350
Created a chunk of size 591, which is longer than the specified 350
Created a chunk of size 362, which is longer th

In [ ]:
# test query
db.similarity_search("I have paranoia", k = 2)

[Document(page_content='These situations can result in an increase in anxiety or a panic attack or feeling the need to separate themselves from family or friends.People with agoraphobia sometimes fear waiting outside for long periods of time; that symptom can be called "macrophobia"'),
 Document(page_content='People with specific phobias understand that their fear is not proportional to the actual potential danger, but they can still become overwhelmed by it.\n\n\n=== Panic disorder ===\n\nWith panic disorder, a person has brief attacks of intense terror and apprehension, often marked by trembling, shaking, confusion, dizziness, or difficulty breathing')]

In [24]:
llm_chain = LLMChain(prompt=prompt, llm=llm)
question = input(">> ")
past_questions = []

while 'quit' not in question:

    disorder = "anxiety"
    db = FAISS.load_local(f'faiss_databases/{disorder}', OpenAIEmbeddings(api_key = api_key))

    context = ""
    for t in db.similarity_search(question):
        context += t.page_content + "\n"

    memory_content = ""
    for i in past_questions:
        memory_content += i + "\n"

    print(memory_content)

    ans = llm_chain.invoke({'disorder': disorder, 'disorder_context': context, 'survey': "", 'question': question, 'past_qs': memory_content})['text']
    print(ans)

    past_questions.append(question)
    if (len(past_questions) > 5): past_questions.pop(0)

    question = input(">> ")


As a chatbot, I am not able to provide medical advice or treatment plans. However, I can suggest seeking help from a mental health professional, such as a therapist or psychiatrist. They can evaluate your specific symptoms and circumstances and work with you to determine the best course of treatment, whether it be therapy, medication, or a combination of both. It's important to communicate openly and honestly with your doctor and to not give up if the first treatment option doesn't work. Remember, seeking help is a brave and important step towards managing your anxiety.
What do I do?
You just asked, "What do I do?"
What do I do?What did I just ask?

There are several ways to deal with anxiety, especially if you are at risk for anxiety disorders. One helpful strategy is to practice stress management techniques, such as meditation and visualization. These practices can help calm your mind and enhance the effects of therapy. Another helpful strategy is to engage in service to others, as 